In [1]:
import dask.dataframe as dd
from dask.distributed import Client

In [2]:
client = Client(
    n_workers=4,
    threads_per_worker=2,
    memory_limit='32GB'
)

c:\Users\dmitrii.druzhbin\AppData\Local\miniconda3\envs\snowcast\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 61797 instead
  warnings.warn(


In [3]:
data_repo = "./data/raw/QUOT_SIM2_previous-2020-202507.csv.gz"
df = dd.read_csv(data_repo ,sep=";",  blocksize=None, assume_missing=True)

In [4]:
df["DATE"] = dd.to_datetime(df["DATE"], format="%Y%m%d", errors="coerce")

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("./data/raw/QUOT_SIM2_previous-2020-202507.csv.gz", sep=';', compression='gzip')

In [4]:
df = df[['LAMBX', 'LAMBY', 'DATE', 'HTEURNEIGE_Q']]

In [5]:
df['DATE'] = pd.to_datetime(df['DATE'], format='%Y%m%d')

In [15]:
df['coords'] = df['LAMBX'].astype(str)+' '+df['LAMBY'].astype(str)

C:\Users\dmitrii.druzhbin\AppData\Local\Temp\ipykernel_14200\845481309.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['coords'] = df['LAMBX'].astype(str)+' '+df['LAMBY'].astype(str)


In [6]:
coords_df = df[['LAMBX', 'LAMBY']]

In [7]:
coords_df = coords_df.rename(columns={'LAMBX':'X','LAMBY':'Y'})

In [8]:
coords_df

,X,Y
0,600,24010
1,600,24010
2,600,24010
3,600,24010
4,600,24010
...,...,...
20169783,11960,17450
20169784,11960,17450
20169785,11960,17450
20169786,11960,17450


In [9]:
import geopandas as gpd
from shapely.geometry import Point

# Load French departments GeoJSON and extract Isère (code 38)
departments_url = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
departments = gpd.read_file(departments_url)
isere = departments[departments["code"] == "38"].to_crs("EPSG:27572")  # Match Lambert II étendu

# Create GeoDataFrame from coords
geometry = [Point(x, y) for x, y in zip(coords_df['X']*100, coords_df['Y']*100)]
gdf_coords = gpd.GeoDataFrame(coords_df, geometry=geometry, crs="EPSG:27572")

# Filter to keep only points within Isère
gdf_coords_isere = gdf_coords[gdf_coords.within(isere.unary_union)]
gdf_coords_isere = gdf_coords_isere.drop(columns="geometry")

gdf_coords_isere.head()

C:\Users\dmitrii.druzhbin\AppData\Local\Temp\ipykernel_10196\2276140119.py:14: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  gdf_coords_isere = gdf_coords[gdf_coords.within(isere.unary_union)]


,X,Y
1249066,7960,20410
1249067,7960,20410
1249068,7960,20410
1249069,7960,20410
1249070,7960,20410


In [10]:
common_idx = df.index.intersection(gdf_coords_isere.index)
big_filtered = df.loc[common_idx]

In [11]:
out = df.loc[common_idx].join(gdf_coords_isere, how='inner', lsuffix='_big', rsuffix='_small')


In [12]:
out = out.drop(columns=['LAMBX', 'LAMBY'])

In [13]:
out = out.set_index('DATE')

In [15]:
from pyproj import Transformer
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Step 1: Convert Lambert II étendu (EPSG:27572) X/Y to WGS84 lat/lon
transformer = Transformer.from_crs("EPSG:27572", "EPSG:4326", always_xy=True)

In [16]:
out['lon'], out['lat'] = transformer.transform((out['X']*100).values, (out['Y']*100).values)

In [17]:
out = out.reset_index()

In [18]:
out['month'] =  out['DATE'].dt.to_period('M')

In [19]:
out

,DATE,HTEURNEIGE_Q,X,Y,lon,lat,month
0,2020-01-01,0.000,7960,20410,4.837191,45.341433,2020-01
1,2020-01-02,0.000,7960,20410,4.837191,45.341433,2020-01
2,2020-01-03,0.000,7960,20410,4.837191,45.341433,2020-01
3,2020-01-04,0.000,7960,20410,4.837191,45.341433,2020-01
4,2020-01-05,0.000,7960,20410,4.837191,45.341433,2020-01
...,...,...,...,...,...,...,...
248753,2025-07-27,0.000,9160,20010,6.340070,44.937265,2025-07
248754,2025-07-28,0.001,9160,20010,6.340070,44.937265,2025-07
248755,2025-07-29,0.000,9160,20010,6.340070,44.937265,2025-07
248756,2025-07-30,0.000,9160,20010,6.340070,44.937265,2025-07


In [20]:
monthly = out.groupby(['month', 'lon', 'lat'])['HTEURNEIGE_Q'].mean().reset_index()
monthly['month'] = monthly['month'].astype(str)

In [21]:
monthly

,month,lon,lat,HTEURNEIGE_Q
0,2020-01,4.837191,45.341433,0.000000
1,2020-01,4.840441,45.413365,0.000000
2,2020-01,4.843699,45.485298,0.000000
3,2020-01,4.846966,45.557232,0.000000
4,2020-01,4.939185,45.339098,0.000000
...,...,...,...,...
8169,2025-07,6.167298,45.375680,0.000000
8170,2025-07,6.233863,44.869013,0.000387
8171,2025-07,6.238884,44.940882,0.000097
8172,2025-07,6.334919,44.865401,0.000710


In [26]:
import plotly.express as px
import json
import urllib.request

# Create the map with scatter_map
fig = px.scatter_map(
    monthly,
    lat="lat",
    lon="lon",
    size="HTEURNEIGE_Q",
    color="HTEURNEIGE_Q",
    animation_frame="month",
    size_max=20,
    color_continuous_scale="Blues",  # More vibrant than 'Blues'
    range_color=[-0.2, 1],
    title="Monthly Mean Snow Depth (m) in Isère",
    zoom=7,
    center={"lat": 45.3, "lon": 5.6},
    map_style="carto-positron"  # Same style, works with MapLibre
    
)
fig.update_layout(height=700, margin={"r":0,"t":50,"l":0,"b":0})

# Set marker symbol to squares
# fig.update_traces(marker=dict(symbol="square"))

# Add Isère boundary using GeoJSON
url = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
with urllib.request.urlopen(url) as response:
    isere_geojson = json.load(response)

isere_feature = [f for f in isere_geojson["features"] if f["properties"]["code"] == "38"]
isere_geojson = {"type": "FeatureCollection", "features": isere_feature}

# Add the boundary as a layer
fig.update_layout(
    map_layers=[
        {
            "source": isere_geojson,
            "type": "line",
            "color": "black",
            "line": {"width": 1.5}
        }
    ]
)

fig.show()